# 1. Energy data

# M = 15

## 1.0 Startup

In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [2]:
dataset = 'energy_data'
load ('./data/energy_data.mat');
[ndata, ~] = size(M);
X = M(:,[1:25,27:28]);
Y = M(:,26);
R = randperm(ndata);        
xt = X(R(1:1735),:);
yt = Y(R(1:1735));
R(1:1735) = [];
x = X(R,:);          
y = Y(R);
n = 18000; nt = 1735;   %Ms = 15

dataset = energy_data


In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'Y' ; opts.Ynorm = 'Y' ;
opts.Ms = 15 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

In [4]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/energymodel15', 'models','t_dGP_train')

Linesearch     25;  Value -5.509322e+04


partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/energymodel30');
models = s.models;


## 1.1 RBCM vs TERBCM

### 1.1.1 RBCM baseline

In [5]:
criterions = {'RBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans RBCM
RBCM: SMSE 0.0009, MSLL -2.4782, NLPD 0.3810


### 1.1.2 Grid q

In [6]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/terbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing energy_data kmeans TERBCM
TERBCM: SMSE 0.0009, MSLL -2.5589, NLPD 0.3003 - q   0.20
TERBCM: SMSE 0.0009, MSLL -3.2081, NLPD -0.3489 - q   0.40
TERBCM: SMSE 0.0009, MSLL -3.2769, NLPD -0.4177 - q   0.60
TERBCM: SMSE 0.0009, MSLL -3.0442, NLPD -0.1850 - q   0.80
TERBCM: SMSE 0.0009, MSLL -2.4782, NLPD 0.3810 - q   1.00
TERBCM: SMSE 0.0008, MSLL -1.4019, NLPD 1.4574 - q   1.20
TERBCM: SMSE 0.0008, MSLL 0.5506, NLPD 3.4099 - q   1.40
TERBCM: SMSE 0.0008, MSLL 4.0735, NLPD 6.9328 - q   1.60
TERBCM: SMSE 0.0008, MSLL 10.4762, NLPD 13.3354 - q   1.80
TERBCM: SMSE 0.0008, MSLL 22.2496, NLPD 25.1088 - q   2.00
TERBCM: SMSE 0.0008, MSLL 44.1905, NLPD 47.0498 - q   2.20
TERBCM: SMSE 0.0008, MSLL 85.6442, NLPD 88.5034 - q   2.40
TERBCM: SMSE 0.0008, MSLL 165.0108, NLPD 167.8700 - q   2.60
TERBCM: SMSE 0.0008, MSLL 318.8632, NLPD 321.7224 - q   2.80
TERBCM: SMSE 0.0008, MSLL 620.5082, NLPD 623.3675 - q   3.00


### 1.1.3 Find single q

In [7]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=50, bs=100, lr=0.0001, lambda=10, init_q=1.5);

Optimizing q
Optimizing q: remain iter - 50, q - 1.5000
grad_q_norm =   -1.3176e+05
grad_q_reg = -576.28
Optimizing q: remain iter - 49, q - 1.4902
grad_q_norm = -393.68
grad_q_reg = -84.041
Optimizing q: remain iter - 48, q - 1.4901
grad_q_norm =  613.12
grad_q_reg = -220.73
Optimizing q: remain iter - 47, q - 1.4900
grad_q_norm = -2079.8
grad_q_reg = -52.890
Optimizing q: remain iter - 46, q - 1.4898
grad_q_norm = -4962.7
grad_q_reg = -49.160
Optimizing q: remain iter - 45, q - 1.4894
grad_q_norm =   -9.8582e+04
grad_q_reg = -483.71
Optimizing q: remain iter - 44, q - 1.4820
grad_q_norm = -7342.2
grad_q_reg = -316.94
Optimizing q: remain iter - 43, q - 1.4813
grad_q_norm =   -1.0777e+05
grad_q_reg = -620.19
Optimizing q: remain iter - 42, q - 1.4731
grad_q_norm = -1778.3
grad_q_reg = -244.10
Optimizing q: remain iter - 41, q - 1.4728
grad_q_norm =   -1.1412e+04
grad_q_reg = -261.09
Optimizing q: remain iter - 40, q - 1.4718
grad_q_norm =   -3.4018e+04
grad_q_reg = -394.59
Optimizing 

In [8]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.1.4 Find multi q

In [9]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=50, bs=100, lr=0.001, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Optimizing qs: remain iter - 50
qs =

   1.0221
   1.1595
   1.7694
   1.7011
   1.0222
   1.3922
   1.1980
   1.1745
   1.0804
   1.9098
   1.9486
   1.5800
   1.1631
   1.2306
   1.6717

ttb =    1.0513e+05
Optimizing qs: remain iter - 49
qs =

   1.0225
   1.1596
   1.7694
   1.7013
   1.0222
   1.3921
   1.1981
   1.1745
   1.0804
   1.9098
   1.9486
   1.5807
   1.1635
   1.2309
   1.6718

ttb =    1.5261e+05
Optimizing qs: remain iter - 48
qs =

   1.0224
   1.1596
   1.7694
   1.7013
   1.0222
   1.3921
   1.1980
   1.1744
   1.0795
   1.9098
   1.9396
   1.5807
   1.1635
   1.2295
   1.6718

ttb =    1.5152e+05
Optimizing qs: remain iter - 47
qs =

   1.0222
   1.1596
   1.7694
   1.7013
   1.0222
   1.3921
   1.1980
   1.1744
   1.0779
   1.9097
   1.9395
   1.5807
   1.1635
   1.2266
   1.6718

ttb =    1.3541e+05
Optimizing qs: remain iter - 46
qs =

   1.0218
   1.1588
   1.7694
   1.6997
   1.0223
   1.3731
   1.1292
   1.1745
   1.0779
   1.9097
   1.9395
   

In [10]:
qs
save('./res/terbcmenergydataqs', 'qs');
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.00967
   1.03710
   0.96424
   1.08229
   0.23567
   0.72900
   0.77603
   0.88388
   1.05887
   1.90372
   1.12748
   0.68808
   0.34999
   1.15418
   0.85359

TERBCM: SMSE 0.0009, MSLL -0.9785, NLPD 1.8807


In [11]:
save('./res/terbcmenergydataqs_res', 'SMSE','MSLL','NLPD')

## 1.2 GRBCM vs TEGRBCM

### 1.2.1 GRBCM baseline

In [12]:
criterions = {'GRBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GRBCM
GRBCM: SMSE 0.0010, MSLL -4.8723, NLPD -2.0130


### 1.2.2 Grid q

In [13]:
criterions = {'TEGRBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegrbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing energy_data kmeans TEGRBCM
TEGRBCM: SMSE 0.0019, MSLL -4.5681, NLPD -1.7088 - q   0.20
TEGRBCM: SMSE 0.0017, MSLL -4.6264, NLPD -1.7672 - q   0.40
TEGRBCM: SMSE 0.0014, MSLL -4.7156, NLPD -1.8564 - q   0.60
TEGRBCM: SMSE 0.0012, MSLL -4.8113, NLPD -1.9521 - q   0.80
TEGRBCM: SMSE 0.0010, MSLL -4.8723, NLPD -2.0130 - q   1.00
TEGRBCM: SMSE 0.0009, MSLL -4.8125, NLPD -1.9532 - q   1.20
TEGRBCM: SMSE 0.0009, MSLL -4.4582, NLPD -1.5990 - q   1.40
TEGRBCM: SMSE 0.0010, MSLL -3.4628, NLPD -0.6035 - q   1.60
TEGRBCM: SMSE 0.0011, MSLL -1.1329, NLPD 1.7263 - q   1.80
TEGRBCM: SMSE 0.0012, MSLL 3.9289, NLPD 6.7882 - q   2.00
TEGRBCM: SMSE 0.0014, MSLL 14.5710, NLPD 17.4302 - q   2.20
TEGRBCM: SMSE 0.0015, MSLL 36.6507, NLPD 39.5099 - q   2.40
TEGRBCM: SMSE 0.0015, MSLL 82.2975, NLPD 85.1568 - q   2.60
TEGRBCM: SMSE 0.0016, MSLL 176.7589, NLPD 179.6182 - q   2.80
TEGRBCM: SMSE 0.0016, MSLL 372.8150, NLPD 375.6742 - q   3.00


### 1.2.3 Find single q

In [37]:
criterion = 'TEGRBCM';

In [15]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=50, bs=100, lr=0.001, lambda=10, init_q=1.5);

Optimizing q
Optimizing q: remain iter - 50, q - 1.5000
grad_q_norm =    1.0168e+04
grad_q_reg = -55.431
Optimizing q: remain iter - 49, q - 1.5069
grad_q_norm =   -2.1910e+05
grad_q_reg = -51.391
Optimizing q: remain iter - 48, q - 1.3500
grad_q_norm =   -1.1050e+04
grad_q_reg = -10.951
Optimizing q: remain iter - 47, q - 1.3420
grad_q_norm =   -9.0835e+06
grad_q_reg = -36.619
Optimizing q: remain iter - 46, q - 0.5420
grad_q_norm =   -1.8270e+04
grad_q_reg = -0.27905
Optimizing q: remain iter - 45, q - 0.5290
grad_q_norm = -82.831
grad_q_reg =  0.10761
Optimizing q: remain iter - 44, q - 0.5289
grad_q_norm = -16.481
grad_q_reg =  0.11377
Optimizing q: remain iter - 43, q - 0.5289
grad_q_norm = -661.60
grad_q_reg = -0.20531
Optimizing q: remain iter - 42, q - 0.5284
grad_q_norm = -1662.9
grad_q_reg = -0.099145
Optimizing q: remain iter - 41, q - 0.5272
grad_q_norm = -1530.4
grad_q_reg = -0.076307
Optimizing q: remain iter - 40, q - 0.5262
grad_q_norm = -48.775
grad_q_reg =  0.022366
O

In [16]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.2.4 Find multi q

In [38]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=50, bs=100, lr=0.001, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Optimizing qs: remain iter - 50
qs =

   1.4446
   1.1303
   1.1216
   1.4296
   1.7234
   1.0362
   1.4375
   1.6736
   1.4493
   1.6774
   1.6888
   1.6222
   1.6916
   1.0586
   1.3098

ttb =    1.1309e+04
Optimizing qs: remain iter - 49
qs =

   1.4446
   1.1303
   1.0393
   1.4195
   1.7137
   1.0357
   1.4373
   1.6741
   1.4492
   1.6777
   1.6891
   1.6098
   1.6788
   1.0586
   1.3084

ttb =  5722.4
Optimizing qs: remain iter - 48
qs =

   1.4446
   1.1295
   1.0392
   1.4138
   1.7112
   1.0136
   1.4373
   1.6743
   1.4469
   1.6780
   1.6894
   1.6062
   1.6711
   1.0577
   1.3077

ttb =    2.3301e+04
Optimizing qs: remain iter - 47
qs =

   1.44463
   1.12822
   1.03927
   1.41106
   1.71339
   0.98924
   1.42333
   1.67212
   1.44180
   1.67830
   1.68684
   1.60879
   1.67112
   1.05810
   1.30669

ttb =    2.1354e+04
Optimizing qs: remain iter - 46
qs =

   1.44463
   1.11439
   1.03927
   1.41109
   1.71342
   0.98925
   1.42327
   1.67201
   1.43937
   1.

   0.94860
   0.79659
   1.14976
   1.47645
   1.41090
   1.45327
   1.49289
   1.04761
   1.21446

ttb =  1428.9
Optimizing qs: remain iter - 12
qs =

   1.44463
   0.87791
   1.02348
   1.14092
   0.91083
   0.69529
   0.94860
   0.79660
   1.14965
   1.47670
   1.41083
   1.44818
   1.48705
   1.04758
   1.21441

ttb =  6849.4
Optimizing qs: remain iter - 11
qs =

   1.44463
   0.87713
   1.02349
   1.14101
   0.91093
   0.69531
   0.94858
   0.79624
   1.14947
   1.47672
   1.41035
   1.44818
   1.48752
   1.04765
   1.21438

ttb =  2007.3
Optimizing qs: remain iter - 10
qs =

   1.44463
   0.87698
   1.02358
   1.14101
   0.91105
   0.69529
   0.94827
   0.79624
   1.14914
   1.47667
   1.40889
   1.44812
   1.48770
   1.04766
   1.21431

ttb =  5778.3
Optimizing qs: remain iter - 9
qs =

   1.44463
   0.87689
   1.02363
   1.14107
   0.91107
   0.69530
   0.94832
   0.79627
   1.14895
   1.44362
   1.40889
   1.44813
   1.48780
   1.04772
   1.21437

ttb =  1631.4
Optimizing qs: 

In [39]:
qs
save('./res/tegrbcmenergydataqs', 'qs');
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.44463
   0.66191
   1.01736
   1.10945
   0.91066
   0.66606
   0.40035
   0.24288
   0.93515
   1.33622
   1.23075
   1.41717
   1.45324
   1.01996
   1.18313

TEGRBCM: SMSE 0.0011, MSLL -4.6511, NLPD -1.7919


In [40]:
save('./res/tegrbcmenergydataqs_res', 'SMSE','MSLL','NLPD')

## 1.3 GPoE vs TEGPoE

### 1.3.1 GPoE baseline

In [20]:
criterions = {'GPoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GPoE
GPoE: SMSE 0.0009, MSLL -2.4565, NLPD 0.4028


### 1.3.2 Grid q

In [21]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TEGPoE'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegpoe%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing energy_data kmeans TEGPoE
TEGPoE: SMSE 0.0009, MSLL -2.5051, NLPD 0.3542 - q   0.20
TEGPoE: SMSE 0.0009, MSLL -3.1712, NLPD -0.3120 - q   0.40
TEGPoE: SMSE 0.0009, MSLL -3.2476, NLPD -0.3884 - q   0.60
TEGPoE: SMSE 0.0009, MSLL -3.0195, NLPD -0.1603 - q   0.80
TEGPoE: SMSE 0.0009, MSLL -2.4565, NLPD 0.4028 - q   1.00
TEGPoE: SMSE 0.0008, MSLL -1.3820, NLPD 1.4773 - q   1.20
TEGPoE: SMSE 0.0008, MSLL 0.5696, NLPD 3.4288 - q   1.40
TEGPoE: SMSE 0.0008, MSLL 4.0925, NLPD 6.9518 - q   1.60
TEGPoE: SMSE 0.0008, MSLL 10.4962, NLPD 13.3555 - q   1.80
TEGPoE: SMSE 0.0008, MSLL 22.2719, NLPD 25.1311 - q   2.00
TEGPoE: SMSE 0.0008, MSLL 44.2165, NLPD 47.0757 - q   2.20
TEGPoE: SMSE 0.0008, MSLL 85.6754, NLPD 88.5346 - q   2.40
TEGPoE: SMSE 0.0008, MSLL 165.0491, NLPD 167.9083 - q   2.60
TEGPoE: SMSE 0.0008, MSLL 318.9100, NLPD 321.7693 - q   2.80
TEGPoE: SMSE 0.0008, MSLL 620.5634, NLPD 623.4227 - q   3.00


### 1.3.3 Find single q

In [22]:
criterion = 'TEGPoE';

In [23]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=150, bs=100, lr=0.001, lambda=10, init_q=1.5);

Optimizing q
Optimizing q: remain iter - 150, q - 1.5000
grad_q_norm = -3903.8
grad_q_reg = -174.11
Optimizing q: remain iter - 149, q - 1.4960
grad_q_norm =   -1.5097e+06
grad_q_reg = -682.94
Optimizing q: remain iter - 148, q - 0.6960
grad_q_norm =  2093.6
grad_q_reg =  6.5071
Optimizing q: remain iter - 147, q - 0.6975
grad_q_norm =  276.57
grad_q_reg =  7.8312
Optimizing q: remain iter - 146, q - 0.6978
grad_q_norm =   -3.7685e+05
grad_q_reg =  3.2495
Optimizing q: remain iter - 145, q - 0.4286
grad_q_norm =    2.3077e+04
grad_q_reg =  57.508
Optimizing q: remain iter - 144, q - 0.4455
grad_q_norm =    4.7370e+04
grad_q_reg =  48.796
Optimizing q: remain iter - 143, q - 0.4797
grad_q_norm =    1.1567e+04
grad_q_reg =  40.119
Optimizing q: remain iter - 142, q - 0.4882
grad_q_norm =  980.24
grad_q_reg =  28.908
Optimizing q: remain iter - 141, q - 0.4891
grad_q_norm =  496.85
grad_q_reg =  32.169
Optimizing q: remain iter - 140, q - 0.4897
grad_q_norm =  4184.9
grad_q_reg =  28.482


In [24]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.3.4 Find multi q

In [30]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=50, bs=100, lr=0.001, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Optimizing qs: remain iter - 50
qs =

   1.0680
   1.9670
   1.6156
   1.1619
   1.6723
   1.9147
   1.9069
   1.7667
   1.7397
   1.9560
   1.8783
   1.9924
   1.2702
   1.6047
   1.4603

ttb = 0
Optimizing qs: remain iter - 49
qs =

   1.0674
   1.9671
   1.6155
   1.1619
   1.6722
   1.9142
   1.9065
   1.7667
   1.7396
   1.9560
   1.8783
   1.9923
   1.2702
   1.6045
   1.4576

ttb = 0
Optimizing qs: remain iter - 48
qs =

   1.0670
   1.1737
   1.6155
   1.1618
   1.6722
   1.9055
   1.9065
   1.7661
   1.7344
   1.9560
   1.8782
   1.9921
   1.2702
   1.6042
   1.4575

ttb = 0
Optimizing qs: remain iter - 47
qs =

   1.0670
   1.1737
   1.6155
   1.1618
   1.6722
   1.9055
   1.9065
   1.7661
   1.7344
   1.9236
   1.8782
   1.9921
   1.2702
   1.6042
   1.4575

ttb = 0
Optimizing qs: remain iter - 46
qs =

   1.0665
   1.1739
   1.6155
   1.1618
   1.6722
   1.9055
   1.2162
   1.7661
   1.7337
   1.9236
   1.8783
   1.9915
   1.2695
   1.6041
   1.4575

ttb = 0
Op

   0.29160

ttb = 0
Optimizing qs: remain iter - 10
qs =

   1.04087
   0.67136
   0.24819
   1.06022
   1.62773
   0.55225
   0.75920
   1.36685
   1.48792
   1.92111
   1.87559
   1.74178
   1.08356
   1.48662
   0.29336

ttb = 0
Optimizing qs: remain iter - 9
qs =

   1.04045
   0.67227
   0.25714
   1.05948
   1.62774
   0.55233
   0.75926
   1.36686
   1.48762
   1.92113
   1.87561
   1.74171
   1.08357
   1.48662
   0.58757

ttb = 0
Optimizing qs: remain iter - 8
qs =

   1.03981
   0.67234
   0.25714
   1.05948
   1.62774
   0.55237
   0.75928
   1.36687
   1.48761
   1.92114
   1.87561
   1.74172
   1.08358
   1.48648
   0.58753

ttb = 0
Optimizing qs: remain iter - 7
qs =

   1.03981
   0.67295
   0.25732
   1.05950
   1.62774
   0.55241
   0.75946
   1.36688
   1.47184
   1.12114
   1.87478
   1.74172
   1.08358
   1.48652
   0.58772

ttb = 0
Optimizing qs: remain iter - 6
qs =

   1.03895
   0.67303
   0.25773
   1.05945
   1.62775
   0.55249
   0.75948
   1.36690
   1.47178

In [31]:
qs
save('./res/tegpoeenergydataqs', 'qs');
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.03573
   0.67463
   0.26143
   1.05618
   1.49790
   0.46937
   0.75870
   1.36469
   1.47189
   1.12116
   1.87486
   1.71643
   0.28078
   1.48455
   0.58908

TEGPoE: SMSE 0.0009, MSLL -0.1061, NLPD 2.7532


In [32]:
save('./res/tegpoeenergydataqs_res', 'SMSE','MSLL','NLPD')